# 01-01 : Basic Classification Test

Test the basic functionality for using a Mistral Model for classifying customer reviews with multiple intents.

## Ollama

To keep things simple the 7B model released by Mistral AI, updated to version 0.2 will be used and hosted locally on the same machine with Ollama. In a production situation this model can be hosted in [AWS SageMaker](https://aws.amazon.com/blogs/machine-learning/mistral-7b-foundation-models-from-mistral-ai-are-now-available-in-amazon-sagemaker-jumpstart/) or a hosted service as provided by [Mistral AI](https://docs.mistral.ai/). 

Linux users can install Ollama with the following command:

```bash
curl -fsSL https://ollama.com/install.sh | sh
```

The following commands are then executed to run Mistral:

```bash
ollama pull mistral
ollama run mistral
```

The following command can be executed to confirm that the model is running as expected:

```bash
curl -X POST http://localhost:11434/api/generate -d '{
  "model": "mistral",
  "prompt":"Here is a story about llamas eating grass"
 }'
```

## References

- [Get up and running with large language models, locally](https://ollama.com/)
- [Mistral with Ollama](https://ollama.com/library/mistral)
- [LangChain Ollama](https://python.langchain.com/docs/integrations/llms/ollama#via-langchain)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append(os.path.abspath("../../src"))

In [3]:
from IPython.display import display, Markdown
from pprint import pprint

## Create LLM 

Create the LangChain client to use the Mistral model.

In [4]:
from langchain_community.llms import Ollama

In [5]:
# create the client
llm = Ollama(
    model="mistral",
    top_p=0.001,
    temperature=0.001,
    num_predict=512)

# ensure that the client is working
Markdown(
    llm.invoke("NASA said the moon is just a large cheese in the sky! 🧀")
)

 I'm an assistant designed to help answer questions and generate text based on given prompts. I don't have the ability to hold beliefs or make statements as if I were NASA. However, I can tell you that NASA has never made such a statement about the moon being a large cheese in the sky. The moon is actually a terrestrial planet, not a dairy product.

## Perform a Classification

The `service.classification.classification` module provides an abstraction the retrieve classifications from a user review.

- The possible categories (intents) are configured in `src/config/category_definitions.jsonl`

- Multiple prompts, similar to a Chain-of-Thought (CoT) approach is used to categorize reviews.
  1. First a general plan of action is composed to categorize the review via `src/config/planning_prompt.txt`.
  2. Next only the main categories are identified: `src/config/identification_prompt.txt`.
  3. Finally structured output is produced identifying the multiple intents and sentiments as shown in the code sample below: `src/config/sentiment_prompt.txt`.

In [6]:
import service.classification as classification

In [7]:
text = "The quick brown fox is experiencing bad network service. However, the lazy dog has fast internet."
result = classification.get_classification(text=text, llm=llm)

pprint(result.dict())

{'categories': [{'category': 'Network Coverage',
                 'reason': "The text mentions 'bad network service' which "
                           'falls under the Network Coverage category.',
                 'relevance': 1.0,
                 'sentiment': 'negative'}]}


In [8]:
text = "MTN really rubs my tits the wrong way with their over billing mistakes."
result = classification.get_classification(text=text, llm=llm)

pprint(result.dict())

{'categories': [{'category': 'Billing',
                 'reason': "The text mentions 'over billing mistakes' by MTN.",
                 'relevance': 1.0,
                 'sentiment': 'negative'},
                {'category': "Customer's Feeling",
                 'reason': "The text expresses the customer's negative emotion "
                           'towards MTN.',
                 'relevance': 1.0,
                 'sentiment': 'negative'}]}
